In [28]:
from WikiArticleXmlHandler import WikiArticleXmlHandler
from WikiDataXmlHandler import WikiDataXmlHandler
import xml.sax
import mwparserfromhell
import stanfordnlp

import subprocess
import pandas as pd
import os
import Utils

from ProcessArticle import ProcessArticle

# WIKIDATA

In [5]:
file = "./1.wikidata/wikidatawiki-latest-pages-articles-multistream1.xml-p1p235321.bz2"

In [6]:
%%time

handler = WikiDataXmlHandler("./askjh.csv")
parser = xml.sax.make_parser()
parser.setContentHandler(handler)

print("Started processing: {}\n".format(file))
for line in subprocess.Popen(["bzcat"],
                        stdin=open(file),
                        stdout=subprocess.PIPE).stdout:
    
    if handler._counter > 5: break
    parser.feed(line)

Started processing: ./1.wikidata/wikidatawiki-latest-pages-articles-multistream1.xml-p1p235321.bz2

Wall time: 421 ms


In [23]:
for i in range(6):
    print(handler._values["title"])

Q22
Q22
Q22
Q22
Q22
Q22


In [10]:
import json

a = json.loads(handler._values["text"])

In [11]:
a.keys()

dict_keys(['type', 'id', 'labels', 'descriptions', 'aliases', 'claims', 'sitelinks'])

In [19]:
a["claims"].keys()

dict_keys(['P2924', 'P1082', 'P1667', 'P1151', 'P1546', 'P5125', 'P349', 'P38', 'P1566', 'P227', 'P3896', 'P1792', 'P2938', 'P2581', 'P2633', 'P1313', 'P3417', 'P417', 'P17', 'P3219', 'P1705', 'P2959', 'P3106', 'P982', 'P94', 'P486', 'P85', 'P418', 'P4212', 'P935', 'P194', 'P37', 'P605', 'P2184', 'P1365', 'P214', 'P300', 'P30', 'P36', 'P856', 'P237', 'P1549', 'P2997', 'P402', 'P1245', 'P691', 'P3241', 'P5905', 'P998', 'P1225', 'P910', 'P3612', 'P35', 'P1791', 'P242', 'P836', 'P1036', 'P2046', 'P3120', 'P6', 'P2163', 'P4427', 'P5573', 'P1417', 'P4839', 'P948', 'P1740', 'P208', 'P1830', 'P1465', 'P1711', 'P2347', 'P3722', 'P1906', 'P1296', 'P373', 'P78', 'P47', 'P31', 'P610', 'P1464', 'P4672', 'P3222', 'P421', 'P3616', 'P5247', 'P150', 'P646', 'P244', 'P163', 'P625', 'P131', 'P1343', 'P41', 'P5019', 'P6573', 'P4801', 'P706', 'P5198', 'P2936', 'P2132', 'P2131', 'P6404', 'P7352', 'P7471'])

In [24]:
next(iter(a["claims"]["P31"]))

{'mainsnak': {'snaktype': 'value',
  'property': 'P31',
  'hash': '74f35128188f12ab9fd1c1948fd0db1ecb98dc76',
  'datavalue': {'value': {'entity-type': 'item',
    'numeric-id': 3336843,
    'id': 'Q3336843'},
   'type': 'wikibase-entityid'}},
 'type': 'statement',
 'id': 'q22$A821BE15-C339-44BC-9DF7-BC573756C882',
 'rank': 'preferred'}

In [ ]:
# get wikidata instanceof
a["claims"]["P31"][0]["mainsnak"]["datavalue"]["value"]["id"]

In [ ]:
for i in range(len(a["claims"]["P31"])):
    print(a["claims"]["P31"][i]["mainsnak"]["datavalue"]["value"]["id"])

In [ ]:
x = "|".join([a["claims"]["P31"][i]["mainsnak"]["datavalue"]["value"]["id"] for i in range(len(a["claims"]["P31"]))])
x

# ARTICLES

In [29]:
infoboxes = pd.read_csv("./2.data/infobox-list.csv")["infobox template name"].values.tolist()
out_file = "./saved/debug.csv"

nlp = stanfordnlp.Pipeline(models_dir="./stanfordnlp_resources/", use_gpu=False);

func = ProcessArticle(infoboxes, nlp=nlp)

Use device: cpu
---
Loading: tokenize
With settings:
{'model_path': './stanfordnlp_resources/en_ewt_models\\en_ewt_tokenizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: pos
With settings:
{'model_path': './stanfordnlp_resources/en_ewt_models\\en_ewt_tagger.pt', 'pretrain_path': './stanfordnlp_resources/en_ewt_models\\en_ewt.pretrain.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: lemma
With settings:
{'model_path': './stanfordnlp_resources/en_ewt_models\\en_ewt_lemmatizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
Building an attentional Seq2Seq model...
Using a Bi-LSTM encoder
Using soft attention for LSTM.
Finetune all embeddings.
[Running seq2seq lemmatizer with edit classifier]
---
Loading: depparse
With settings:
{'model_path': './stanfordnlp_resources/en_ewt_models\\en_ewt_parser.pt', 'pretrain_path': './stanfordnlp_resources/en_ewt_models\\en_ewt.pretrain.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': '

In [30]:
%%time
file = "./test_data/test.xml"

handler = WikiArticleXmlHandler(out_file, func, 1000)
handler._DEBUG = True

parser = xml.sax.make_parser()
parser.setContentHandler(handler)

for line in open(file, "r", encoding="utf8"):
    parser.feed(line)

# handler.write_df()

..\aten\src\ATen\native\LegacyDefinitions.cpp:14: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
..\aten\src\ATen\native\LegacyDefinitions.cpp:14: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
..\aten\src\ATen\native\LegacyDefinitions.cpp:14: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
Wall time: 4.24 s


In [ ]:
n_df = pd.read_csv(out_file, delimiter="\t")

In [ ]:
n_df

In [ ]:
n_df[n_df.title.str.contains("(?i)cobol")]

In [ ]:
for i, a in enumerate(handler._data):
    if a[0] == "List of artificial intelligence projects":
        print("{}: {}".format(i, a[0]))

In [31]:
for i, a in enumerate(handler._data):
    # if a[0] == "COBOL":
    print("{}: {}".format(i, a[0]))

0: Abraham Lincoln
1: Apollo
2: Apollo program


In [91]:
page = handler._data[1]
text = handler._text[1]

In [33]:
page[4].split("|")

['type:Greek',
 'name:Apollo',
 'image:File:Apollo of the Belvedere.jpg',
 'alt:',
 'caption:Apollo Belvedere, c. 120–140 CE',
 'god_of:God of the Sun, light, oracles, knowledge, healing, diseases, music, poetry, songs, dance, archery, herds and flocks,  and protection of young',
 'abode:Mount Olympus',
 'symbol:Lyre, laurel wreath, python, raven, swan, bow and arrows',
 'consort:',
 'parents:Zeus and Leto',
 'siblings:Artemis, Aeacus, Angelos, Aphrodite, Ares, Athena, Dionysus, Eileithyia, Enyo, Eris, Ersa, Hebe, Helen of Troy, Hephaestus, Heracles, Hermes, Minos, Pandia, Persephone, Perseus, Rhadamanthus, the Graces, the Horae, the Litae, the Muses, the Moirai',
 'children:Asclepius, Aristaeus, Corybantes, Hymenaeus, Ialemus, , Apollonis, Borysthenis, Cephisso, Agreus, Amphiaraus, Amphissus, Amphithemis, Anius, Apis, Arabus, Centaurus, Ceos, Chaeron, Chios, Chariclo, Chrysorrhoas, Coronus, Cycnus, Cydon, Delphus, Dorus, Dryops, Eleuther, Epidaurus, Eriopis, Erymanthus, Eurydice, Hect

In [ ]:
infoboxes = pd.read_csv("./saved/infobox-list.csv")["infobox template name"].values.tolist()
cols = ["title", "type", "def words", "infobox_name", "infobox", "categories","wikilinks"]

In [92]:
wiki = mwparserfromhell.parse(text)

In [93]:
a = wiki.strip_code().split("\n")
a = [" ".join(Utils.cleanhtml(x.strip()).split()) for x in a if not x.isspace() and x != ""]
a = a[1] # 1st paragraph
a

'Apollo is one of the most important and complex of the Olympian deities in classical Greek and Roman religion and Greek and Roman mythology. The national divinity of the Greeks, Apollo has been recognized as a god of archery, music and dance, truth and prophecy, healing and diseases, the Sun and light, poetry, and more. He is the son of Zeus and Leto, and the twin brother of Artemis, goddess of the hunt. Seen as the most beautiful god and the ideal of the kouros (ephebe, or a beardless, athletic youth), Apollo is considered to be the most Greek of all the gods. Apollo is known in Greek-influenced Etruscan mythology as Apulu. Krauskopf, I. 2006. "The Grave and Beyond." The Religion of the Etruscans. edited by N. de Grummond and E. Simon. Austin: University of Texas Press. p. vii, p. 73-75.'

In [37]:
nlp = stanfordnlp.Pipeline(models_dir="./stanfordnlp_resources/", use_gpu=False);

Use device: cpu
---
Loading: tokenize
With settings:
{'model_path': './stanfordnlp_resources/en_ewt_models\\en_ewt_tokenizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: pos
With settings:
{'model_path': './stanfordnlp_resources/en_ewt_models\\en_ewt_tagger.pt', 'pretrain_path': './stanfordnlp_resources/en_ewt_models\\en_ewt.pretrain.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: lemma
With settings:
{'model_path': './stanfordnlp_resources/en_ewt_models\\en_ewt_lemmatizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
Building an attentional Seq2Seq model...
Using a Bi-LSTM encoder
Using soft attention for LSTM.
Finetune all embeddings.
[Running seq2seq lemmatizer with edit classifier]
---
Loading: depparse
With settings:
{'model_path': './stanfordnlp_resources/en_ewt_models\\en_ewt_parser.pt', 'pretrain_path': './stanfordnlp_resources/en_ewt_models\\en_ewt.pretrain.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': '

In [94]:
doc = nlp(a)

..\aten\src\ATen\native\LegacyDefinitions.cpp:14: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.


In [95]:
sentence = doc.sentences[0]

In [100]:
sentence.words[12]

<Word index=13;text=deities;lemma=deity;upos=NOUN;xpos=NNS;feats=Number=Plur;governor=9;dependency_relation=obl>

In [ ]:
import datetime

months_choices = list()
for i in range(1,13):
    months_choices.append((i, datetime.date(2008, i, 1).strftime('%B')))

months_choices

In [41]:
import datetime

definitions = []
# no_no = ["punct", "nummod", "appos", "det", "nmod:poss", "cop", "cc", "nsubj", "case"]
no_no = ["punct", "nummod", "appos", "det"]
months = [datetime.date(2008, i, 1).strftime("%B").lower() for i in range(1,13)]

In [98]:
print("index\tlemma\t\t\tgovernor\t\t\ttype\n")
for word in sentence.words:
    index = word.index.rjust(2)
    word_lemma = word.lemma
    governor = sentence.words[word.governor-1].lemma if word.governor > 0 else "root"
    deprel = word.dependency_relation

    if deprel in no_no or word_lemma.lower() in months:
        continue

    print("{}\t{}\t\t\t{}\t\t\t{}".format(index, word_lemma, governor, deprel))

index	lemma			governor			type

 1	Apollo			one			nsubj
 2	be			one			cop
 3	one			root			root
 4	of			important			case
 6	most			important			advmod
 7	important			one			nmod
 8	and			complex			cc
 9	complex			important			conj
10	of			deity			case
12	olympian			deity			compound
13	deity			complex			obl
14	in			religion			case
15	classical			religion			amod
16	Greek			religion			amod
17	and			Roman			cc
18	Roman			Greek			conj
19	religion			deity			nmod
20	and			mythology			cc
21	Greek			mythology			amod
22	and			Roman			cc
23	Roman			Greek			conj
24	mythology			religion			conj


In [111]:
def check(word, be_word):
    index = -1
    target_index = be_word.governor
    while(index != target_index):
        index = word.governor
    
    return index

In [110]:
yes_yes = ["nmod", "compound", "obl"]
be_index = None
be_gov = None

bag = set()
for word in sentence.words:
    index = word.index.rjust(2)
    word_lemma = word.lemma
    governor = sentence.words[word.governor-1].lemma if word.governor > 0 else "root"
    deprel = word.dependency_relation

    if word.lemma == "be":
        be = word
        # bag.add(sentence.words[be_gov - 1].lemma)
        break

# now find the realted words with the be lemma
for word in sentence.words:
    if word.governor == be_gov and word.dependency_relation in ["amod", "conj", "compound"]and word.xpos in ["NNS", "NN"] and word.upos == "NOUN":
        bag.add(word.lemma)

bag

set()

In [109]:
sentence.words[8]

<Word index=9;text=complex;lemma=complex;upos=ADJ;xpos=JJ;feats=Degree=Pos;governor=7;dependency_relation=conj>

In [ ]:
a

In [ ]:
print("{} {} {}".format(be_index, be_gov, be_deprel))

In [ ]:
# POS tags
print(" ". join(["{} ({})".format(word.text, word.upos) for word in sentence.words]))

In [ ]:
for word in sentence.tokens[14].words:
    if word.lemma in ["be"]:
        print("a")

In [ ]:
sentence.tokens[14].text

In [ ]:
print(*[f"index: {word.index.rjust(2)}\tword: {word.text.ljust(11)}\tgovernor index: {word.governor}\tgovernor: {(sentence.words[word.governor-1].text if word.governor > 0 else 'root').ljust(11)}\tdeprel: {word.dependency_relation}" for word in sentence.words], sep='\n')

In [ ]:
print(" ". join(["{} ({} - {})".format(word.text, word.upos, word.feats) for word in sentence.words]))

In [ ]:
sentence.build_dependencies()

In [ ]:
sentence.dependencies[0]

In [ ]:
wiki.filter_templates(matches="softredirect")

In [ ]:
"list of" in wiki.filter_templates(matches="DEFAULTSORT")[0].name.lower()

In [ ]:
# Creating dict
d = {col: [] for col in cols}
df = pd.DataFrame(columns=cols)
# parse page
# wiki = mwparserfromhell.parse(page[1])

In [ ]:
t = [None, "3", "None", "6", None, None,"1"]

In [ ]:
[d[cols[i]].append(t[i]) for i in range(len(t))]

In [ ]:
d

In [ ]:
df

In [ ]:
d

In [ ]:
# page title
# d["title"].append(page[0])
df["title"] = [page[0]]

In [ ]:
# defining article type
if page[2] != None:
    # d["type"].append("redirect")
    df["type"] = ["redirect"]
else:
    # d["type"].append("article")
    df["type"] = ["article"]

In [ ]:
# Getting article categories
# d["categories"].append(
#     "|".join([
#         list(filter(None, x.title.split(":")))[-1]
#         for x in wiki.filter_wikilinks(matches="category")
#     ]))
df["categories"] = ["|".join([
        list(filter(None, x.title.split(":")))[-1]
        for x in wiki.filter_wikilinks(matches="category")
    ])]

In [ ]:
# wikilinks, removing links to Category items
# d["wikilinks"].append("|".join(
#     [link.title.strip_code().strip() for link in wiki.filter_wikilinks() if "Category:" not in link.title.strip_code().strip()]))

df["wikilinks"] = ["|".join(
    [link.title.strip_code().strip() for link in wiki.filter_wikilinks() if "Category:" not in link.title.strip_code().strip()])]

# external links
# df["ext_links"] = "|".join([link.url.strip_code().strip() for link in wiki.filter_external_links()])

In [ ]:
import re

def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
  cleantext = re.sub(cleanr, '', raw_html)
  return cleantext

In [ ]:
# search for infobox template name
box = None
for template in wiki.filter_templates():
    if template.name.strip() in infoboxes:
        box = wiki.filter_templates(matches=template.name.strip())[0]
        # d["infobox_name"].append(template.name.strip())
        df["infobox_name"] = [template.name.strip()]
        break

a = "|".join(["{}:{}".format(param.name.strip_code().strip(),
                            param.value.strip_code(keep_template_params=True).strip()) for param in box.params])
# d["infobox"].append(cleanhtml(a))
df["infobox"] = [cleanhtml(a)]

In [ ]:
df

In [ ]:
df["infobox"][0].split("|")

In [ ]:
df["wikilinks"][0].split("|")

In [ ]:
# mais proximo da solucao definitiva
for param in box.params:
    for x in param.value.filter_templates():
        print([y.value.strip_code() for y in x.params])

In [ ]:
# box.params.strip_code()
for param in box.params:
    # print(param.value.filter_text())
    # print(param.value.filter_wikilinks())
    print(param.value.strip_code(keep_template_params=True).strip())

In [ ]:
df["infobox"][0]